#  Link PyPower

In [1]:
from collections import OrderedDict

import numpy as np
import pandas as pd

import ams
import andes
import pandapower as pp
import pandapower.networks

print(ams.__version__)
print(andes.__version__)

0.5+138.g605d361.dirty
1.8.5


In [2]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
                no_output=True,
                setup=True,
                pflow=True,)

Generating code for 1 models on 8 processes.


In [3]:
net = pandapower.networks.case14()
pp.runpp(net)

In [4]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,40.0,43.557100,-4.982589,1.045
1,0.0,25.075348,-12.725100,1.010
2,0.0,12.730944,-14.220946,1.070
3,0.0,17.623451,-13.359627,1.090


In [5]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.060000,0.000000,-232.393272,16.549301
1,1.045000,-4.982589,-18.300000,-30.857100
2,1.010000,-12.725100,94.200000,-6.075348
3,1.017671,-10.312901,47.800000,-3.900000
4,1.019514,-8.773854,7.600000,1.600000
5,1.070000,-14.220946,11.200000,-5.230944
6,1.061520,-13.359627,0.000000,0.000000
7,1.090000,-13.359627,0.000000,-17.623451
8,1.055932,-14.938521,29.500000,-4.584844
9,1.050985,-15.097288,9.000000,5.800000


In [6]:
# NOTE: get a pypower case
from ams.solver.ipp import load_ppc

ppc = load_ppc(ams.get_case('pypower/case14.py'))

ppc.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost'])

In [7]:
ppc['gen'][:, 11:21]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [8]:
sp = ams.load(ams.get_case('ieee14/ieee14.xlsx'),
            setup=True)

In [9]:
sp.summary()

Method `summary` is included in ANDES System but not supported in AMS System.


In [10]:
from ams.solver.ipp import to_ppc

spc, key_dict, col_dict = sp._ppc, sp._key, sp._col

In [11]:
from ams.solver.pypower.runpf import runpf, rundcpf
from ams.solver.pypower.runopf import runopf, rundcopf
from ams.solver.pypower.runuopf import runuopf, runduopf

In [12]:
r_dcpf = rundcpf(spc)

/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:276: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


In [13]:
r_pf, _ = runpf(ppc)

 -- AC Power Flow (Newton)



In [14]:
r_pf.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost', 'order', 'et', 'success'])

In [15]:
r_pf['order'].keys()

dict_keys(['bus', 'gen', 'branch', 'state', 'int'])

In [16]:
r_pf['bus'][:, 7]

array([1.06      , 1.045     , 1.01      , 1.01767085, 1.01951386,
       1.07      , 1.06151953, 1.09      , 1.05593172, 1.05098463,
       1.05690652, 1.05518856, 1.05038171, 1.03552995])

In [17]:
col_dict['bus'].index('Vm')

7

In [18]:
pd.DataFrame(r_pf['bus'], columns=col_dict['bus'])

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,1.060000,0.000000,0.0,1.0,1.06,0.94
1,2.0,2.0,21.7,12.7,0.0,0.0,1.0,1.045000,-4.982589,0.0,1.0,1.06,0.94
2,3.0,2.0,94.2,19.0,0.0,0.0,1.0,1.010000,-12.725100,0.0,1.0,1.06,0.94
3,4.0,1.0,47.8,-3.9,0.0,0.0,1.0,1.017671,-10.312901,0.0,1.0,1.06,0.94
4,5.0,1.0,7.6,1.6,0.0,0.0,1.0,1.019514,-8.773854,0.0,1.0,1.06,0.94
5,6.0,2.0,11.2,7.5,0.0,0.0,1.0,1.070000,-14.220946,0.0,1.0,1.06,0.94
6,7.0,1.0,0.0,0.0,0.0,0.0,1.0,1.061520,-13.359627,0.0,1.0,1.06,0.94
7,8.0,2.0,0.0,0.0,0.0,0.0,1.0,1.090000,-13.359627,0.0,1.0,1.06,0.94
8,9.0,1.0,29.5,16.6,0.0,19.0,1.0,1.055932,-14.938521,0.0,1.0,1.06,0.94
9,10.0,1.0,9.0,5.8,0.0,0.0,1.0,1.050985,-15.097288,0.0,1.0,1.06,0.94


In [19]:
r_pf['bus'].shape

(14, 13)

In [20]:
r_pf['order']

{'bus': {'e2i': array([ 0.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,
         12., 13.]),
  'i2e': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14.]),
  'status': {'on': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
   'off': array([], dtype=int64)}},
 'gen': {'e2i': array([0, 1, 2, 3, 4]),
  'i2e': array([0, 1, 2, 3, 4]),
  'status': {'on': array([0, 1, 2, 3, 4]), 'off': array([], dtype=int64)}},
 'branch': {'status': {'on': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
          17, 18, 19]),
   'off': array([], dtype=int64)}},
 'state': 'e',
 'int': {'bus': array([[  0.        ,   3.        ,   0.        ,   0.        ,
            0.        ,   0.        ,   1.        ,   1.06      ,
            0.        ,   0.        ,   1.        ,   1.06      ,
            0.94      ],
         [  1.        ,   2.        ,  21.7       ,  12.7       ,
            0.        ,   0.        ,   1. 

In [21]:
r_pf

{'version': '2',
 'baseMVA': 100.0,
 'bus': array([[  1.        ,   3.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   1.        ,   1.06      ,
           0.        ,   0.        ,   1.        ,   1.06      ,
           0.94      ],
        [  2.        ,   2.        ,  21.7       ,  12.7       ,
           0.        ,   0.        ,   1.        ,   1.045     ,
          -4.98258914,   0.        ,   1.        ,   1.06      ,
           0.94      ],
        [  3.        ,   2.        ,  94.2       ,  19.        ,
           0.        ,   0.        ,   1.        ,   1.01      ,
         -12.72509994,   0.        ,   1.        ,   1.06      ,
           0.94      ],
        [  4.        ,   1.        ,  47.8       ,  -3.9       ,
           0.        ,   0.        ,   1.        ,   1.01767085,
         -10.31290109,   0.        ,   1.        ,   1.06      ,
           0.94      ],
        [  5.        ,   1.        ,   7.6       ,   1.6       ,
           0.   

In [22]:
r_pf

{'version': '2',
 'baseMVA': 100.0,
 'bus': array([[  1.        ,   3.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   1.        ,   1.06      ,
           0.        ,   0.        ,   1.        ,   1.06      ,
           0.94      ],
        [  2.        ,   2.        ,  21.7       ,  12.7       ,
           0.        ,   0.        ,   1.        ,   1.045     ,
          -4.98258914,   0.        ,   1.        ,   1.06      ,
           0.94      ],
        [  3.        ,   2.        ,  94.2       ,  19.        ,
           0.        ,   0.        ,   1.        ,   1.01      ,
         -12.72509994,   0.        ,   1.        ,   1.06      ,
           0.94      ],
        [  4.        ,   1.        ,  47.8       ,  -3.9       ,
           0.        ,   0.        ,   1.        ,   1.01767085,
         -10.31290109,   0.        ,   1.        ,   1.06      ,
           0.94      ],
        [  5.        ,   1.        ,   7.6       ,   1.6       ,
           0.   

In [23]:
r_dcopf = rundcopf(ppc)

Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


In [24]:
r_opf = runopf(ppc)

Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


In [25]:
r_duopf = runduopf(ppc)

In [26]:
r_uopf = runuopf(ppc)